# SNEAKSAVER IMAGE RECOGINATION TOOL

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers, losses


from keras.models import Sequential
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten, Input, Lambda


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from tensorflow.keras.optimizers import Adam 
import os

In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("")))

train_dir = os.path.join(BASE_DIR, "media", "dataset", "preprocess")
test_dir = os.path.join(BASE_DIR, "media", "dataset", "test")


In [3]:
train_datagen = ImageDataGenerator(rescale = 1./ 255, rotation_range = 40, width_shift_range = 0.2, 
                                   height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, 
                                   horizontal_flip = True, fill_mode = 'nearest')

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./ 255)

In [5]:
train_data = train_datagen.flow_from_directory(directory = train_dir, batch_size = 32, target_size = (240,240), class_mode = "categorical", shuffle = False)
test_data = test_datagen.flow_from_directory(directory = test_dir, batch_size = 32, target_size = (240,240), class_mode = "categorical")

Found 825 images belonging to 3 classes.
Found 114 images belonging to 3 classes.


In [6]:
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation = 'relu', input_shape = (240,240, 3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation = 'relu'),
    Dropout(0.2),
    Dense(3, activation = 'softmax')
])

In [7]:
# Using the Adam Optimizer to set the learning rate of our final model
opt = optimizers.Adam(learning_rate=0.0001)

# Compiling and setting the parameters we want our model to use
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

In [8]:
for image_batch, label_batch in train_data:
  break
image_batch.shape, label_batch.shape

label_batch[0:5]
print (train_data.class_indices)

{'adidas': 0, 'converse': 1, 'nike': 2}


In [9]:
#write all the labels to a text file
labels = "\n".join(sorted(train_data.class_indices.keys()))
with open("labels.txt", "w") as f:
 f.write(labels)
!cat labels.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
history = model.fit(train_data, epochs = 10, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.25 * len(test_data)))

Epoch 1/10

26/26 [==============================] - 408s 13s/step - loss: 1.1866 - accuracy: 0.2230 - val_loss: 1.1093 - val_accuracy: 0.2500
Epoch 2/10
26/26 [==============================] - 301s 12s/step - loss: 1.1234 - accuracy: 0.2461 - val_loss: 1.0988 - val_accuracy: 0.4062
Epoch 3/10
26/26 [==============================] - 331s 12s/step - loss: 1.1082 - accuracy: 0.2727 - val_loss: 1.0928 - val_accuracy: 0.3438
Epoch 4/10
26/26 [==============================] - 230s 9s/step - loss: 1.1024 - accuracy: 0.3321 - val_loss: 1.0883 - val_accuracy: 0.4688
Epoch 5/10
26/26 [==============================] - 170s 6s/step - loss: 1.1077 - accuracy: 0.2545 - val_loss: 1.0838 - val_accuracy: 0.4688
Epoch 6/10
26/26 [==============================] - 184s 7s/step - loss: 1.1140 - accuracy: 0.2521 - val_loss: 1.0957 - val_accuracy: 0.3438
Epoch 7/10
26/26 [==============================] - 166s 6s/step - loss: 1.1007 - accuracy: 0.3309 - val_loss: 1.0921 - val_accuracy: 0.4375
Epoch 8/1

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 238, 238, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 119, 119, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 117, 117, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 58, 58, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 28, 28, 32)        0

In [12]:
# Create a generator for test data
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    batch_size=32,
    target_size=(240, 240),
    class_mode="categorical",
    shuffle=False  # Ensure order is maintained for evaluation
)

# Evaluate the model using the generator
test_loss, test_accuracy = model.evaluate(test_data_generator)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Found 114 images belonging to 3 classes.


4/4 [==============================] - 5s 1s/step - loss: 1.0887 - accuracy: 0.3421
Test Loss: 1.0887137651443481
Test Accuracy: 0.34210526943206787


In [13]:
# Specify the path where you want to save the model
save_path = os.path.join(BASE_DIR, "sneakers_image_recog", "ml_model", 'sneak_image_recog_model.h5')

# Save the model to the specified path
model.save(save_path)

d:\workspace\sneaksaver\sneaksaver-venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Reference

https://www.kaggle.com/code/manarkandeel/sneakers-brand-classification/notebook

In [14]:
softmax_layer = model.layers[-1]

# Extract the class labels from the softmax layer
class_labels = softmax_layer.get_weights()[1]

# Print the class labels
print("Class Labels:", class_labels)

Class Labels: [-0.00020822 -0.0003104   0.00020118]
